In [14]:
# Define Neo4j connections

import pandas as pd
import plotly.express as px
from neo4j import GraphDatabase

In [4]:
# Create Neo4j Connection

host = 'bolt://34.234.223.89:7687'
user = 'neo4j'
password = 'coils-shocks-gages'
driver = GraphDatabase.driver(host, auth=(user, password))

In [6]:
import_queries = """

CALL apoc.schema.assert({Character:['name']},{Comic:['id'], Character:['id'], Event:['id'], Group:['id']});

LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/rrsr28/marvel-bigdataProject/main/data/marvel_scraped/heroes.csv" as row
CREATE (c:Character)
SET c += row;

LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/rrsr28/marvel-bigdataProject/main/data/marvel_scraped/groups.csv" as row
CREATE (c:Group)
SET c += row;

LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/rrsr28/marvel-bigdataProject/main/data/marvel_scraped/events.csv" as row
CREATE (c:Event)
SET c += row;

LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/rrsr28/marvel-bigdataProject/main/data/marvel_scraped/comics.csv" as row
CREATE (c:Comic)
SET c += apoc.map.clean(row,[],["null"]);

LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/rrsr28/marvel-bigdataProject/main/data/marvel_scraped/heroToComics.csv" as row
MATCH (c:Character{id:row.hero})
MATCH (co:Comic{id:row.comic})
MERGE (c)-[:APPEARED_IN]->(co);

LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/rrsr28/marvel-bigdataProject/main/data/marvel_scraped/heroToEvent.csv" as row
MATCH (c:Character{id:row.hero})
MATCH (e:Event{id:row.event})
MERGE (c)-[:PART_OF_EVENT]->(e);

LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/rrsr28/marvel-bigdataProject/main/data/marvel_scraped/heroToGroup.csv" as row
MATCH (c:Character{id:row.hero})
MATCH (g:Group{id:row.group})
MERGE (c)-[:PART_OF_GROUP]->(g);

LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/rrsr28/marvel-bigdataProject/main/data/marvel_scraped/heroToHero.csv" as row
MATCH (s:Character{id:row.source})
MATCH (t:Character{id:row.target})
CALL apoc.create.relationship(s,row.type, {}, t) YIELD rel
RETURN distinct 'done';

LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/rrsr28/marvel-bigdataProject/main/data/marvel_scraped/heroStats.csv" as row
MATCH (s:Character{id:row.hero})
CREATE (s)-[:HAS_STATS]->(stats:Stats)
SET stats += apoc.map.clean(row,['hero'],[]);

LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/rrsr28/marvel-bigdataProject/main/data/marvel_scraped/heroFlight.csv" as row
MATCH (s:Character{id:row.hero})
SET s.flight = row.flight;

MATCH (s:Stats)
WITH keys(s) as keys LIMIT 1
MATCH (s:Stats)
UNWIND keys as key
CALL apoc.create.setProperty(s, key, toInteger(s[key]))
YIELD node
RETURN distinct 'done';
"""

In [ ]:
# Establish a session with the Neo4j database using the provided driver
# Split the import_queries string into individual statements using ';' as the delimiter
# Execute each statement within the session context after stripping leading and trailing whitespace

with driver.session() as session:
    for statement in import_queries.split(';'):
        try:
            session.run(statement.strip())
        except:
            pass

## Run Query

In [8]:
def run_query(query):
    with driver.session() as session:
        result = session.run(query)
        return pd.DataFrame([r.values() for r in result], columns=result.keys())

## Exploratory graph analysis

In [12]:
# Nummber of comic appearances 

run_query("""
            MATCH (c:Character)
            RETURN c.name as character, 
                   count{ (c)-[:APPEARED_IN]->() } as comics
            ORDER BY comics DESC
            LIMIT 10
                        """)

,character,comics
0,Spider-Man (1602),3357
1,Tony Stark,2354
2,Logan,2098
3,Steve Rogers,2019
4,Thor (Marvel: Avengers Alliance),1547
5,Hulk-dok,1433
6,Johnny Storm,1151
7,Matthew Murdock,999
8,Cyclops,851
9,Thing (Marvel Heroes),842


In [26]:
# Call the 'run_query' function with the Cypher query
result_df = run_query("""
                        MATCH (c:Character)
                        RETURN c.name as character, 
                               count{ (c)-[:APPEARED_IN]->() } as comics
                        ORDER BY comics DESC
                        LIMIT 10
                                    """)

print(result_df)
fig = px.bar(result_df, x='character', y='comics', title='Top 10 Characters by Comic Appearances')
fig.update_layout(xaxis_title='Character', yaxis_title='Number of Comics')
fig.show()

                          character  comics
0                 Spider-Man (1602)    3357
1                        Tony Stark    2354
2                             Logan    2098
3                      Steve Rogers    2019
4  Thor (Marvel: Avengers Alliance)    1547
5                          Hulk-dok    1433
6                      Johnny Storm    1151
7                   Matthew Murdock     999
8                           Cyclops     851
9             Thing (Marvel Heroes)     842


In [19]:
# Comic releases through the years

run_query("""
                MATCH (c:Comic)
                RETURN substring(c.year, 0, 3) + "0" as decade, 
                       count(*) as count
                ORDER BY decade ASC
                                        """)

,decade,count
0,1930,95
1,1940,584
2,1950,756
3,1960,4114
4,1970,1956
5,1980,2428
6,1990,3738
7,2000,8309
8,2010,11139
9,2020,19


In [ ]:
result_df = run_query("""
                MATCH (c:Comic)
                RETURN substring(c.year, 0, 3) + "0" as decade, 
                       count(*) as count
                ORDER BY decade ASC
                                        """)

fig = px.bar(result_df, x='decade', y='count', title='Comic Releases Through the Years in Decades')
fig.update_layout(xaxis_title='Decade', yaxis_title='Number of Comic Releases')
fig.show()